In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent.parent)) ## ../../covid19datascience* <- set parent level here

In [ ]:
import pandas as pd
from epiweeks import Week
from datetime import timedelta, date
from os.path import join
import gc

from bulletin import default_input, default_output, root, hoje, data_comeco_pandemia
from bulletin.services.metabase import Metabase
from bulletin.systems.notifica import Notifica
from bulletin.utils.normalize import trim_overspace

In [ ]:
def generate_table(todos_exames):
    tabela = todos_exames.groupby(['SE','classificacao_final'])[['id']].count().unstack()
    
    tabela = tabela.droplevel(0, axis=1)
    
    for row in tabela.index.values:
        if row.year < 2020:
            tabela = tabela.drop(row)
            
    try:
        tabela[5] = tabela[5].add(tabela[0])
        tabela = tabela.drop(0, axis=1)
    except:
        pass
    
    
    tabela = tabela[[2, 3, 1, 5]]
    tabela = tabela.rename(columns={2:'Casos confirmados', 3:'Casos descartados', 1:'Casos suspeitos', 5:'Não informados'})
    
    
    tabela['Total testes'] = tabela.sum(1)
    tabela = tabela.append(pd.DataFrame(tabela.sum(0)).transpose())
    tabela['Positividade (%)'] = tabela['Casos confirmados'].div(tabela['Total testes'])
    
    
    tabela.index.name = 'SE'
    tabela.iloc[-1].name = 'Total Geral'
    
#     tabela.to_excel(join(default_input,'semanais','TABELA_notifica_INPUT.xlsx'))
#     tabela.to_excel('output/TABELA_testes_notifica_RS.xlsx')
    
    
    tabela['Semana Epidemiológica'] = [ x.week for x in tabela.index.values[:-1]] + ['Total Geral']
    tabela = tabela.set_index('Semana Epidemiológica')
    
    tabela.columns.name = None
    tabela.reset_index(inplace=True)
    
    for j in tabela.columns:
        tabela[j] = tabela[j].fillna(0)
    
    return tabela

In [ ]:
#12ª RS - Umuarama
# list_IBGE_RS = [412862,410070,410050,410337,410347,410660,410725,410752,410832,410990,411060,411155,411470,411510,411720,411885,411890,412535,412690,412810,412880]

# exames_RS = pd.DataFrame()
# for municipio in list_IBGE_RS:
#     exames_temp = todos_exames.loc[todos_exames['ibge_unidade_notifica'] == municipio]
#     exames_RS = exames_RS.append(exames_temp)
# todos_exames = exames_RS

In [ ]:
notifica = Notifica()
notifica.load('notifica', compress=False)

In [ ]:
# total_exames = notifica.df.loc[~(notifica.df['metodo'].isna())]
# total_exames = total_exames.loc[(total_exames['uf_residencia']==41)]
# total_exames = total_exames.loc[(total_exames['uf_unidade_notifica']==41)]
# total_exames = total_exames.loc[(total_exames['status_notificacao'].isin([1,2,3]))]
# total_exames = total_exames.loc[~(total_exames['ibge_unidade_notifica'].isnull())]
# total_exames = total_exames.loc[~(total_exames['data_notificacao'].isnull())]
# total_exames = total_exames.loc[~total_exames['metodo'].isin([1,3])]
# print('TOTAL DE TESTES REALIZADOS = ', len(total_exames))

In [ ]:
todos_exames = notifica.df.loc[~(notifica.df['metodo'].isna())]
todos_exames = todos_exames.loc[(todos_exames['excluir_ficha']==2)]
todos_exames = todos_exames.loc[(todos_exames['uf_residencia']==41)]
todos_exames = todos_exames.loc[(todos_exames['uf_unidade_notifica']==41)]
todos_exames = todos_exames.loc[(todos_exames['status_notificacao'].isin([1,2]))]
todos_exames = todos_exames.loc[~(todos_exames['metodo']==3)]
todos_exames = todos_exames.loc[~(todos_exames['ibge_unidade_notifica'].isnull())]
todos_exames = todos_exames.loc[~(todos_exames['data_notificacao'].isnull())].copy()

todos_exames['SE'] = todos_exames['data_notificacao'].apply(lambda x: Week.fromdate(x))

In [ ]:
del(notifica.df)
gc.collect()

In [ ]:
geral = generate_table(todos_exames)
geral.to_excel(join(default_output, 'semanais','TABELA_testes_notifica.xlsx'), index=False)
geral

In [ ]:
grupo01 = todos_exames.loc[(todos_exames['idade'] >= 0) & (todos_exames['idade'] <= 9)]

tabela_grupo01 = generate_table(grupo01)
tabela_grupo01.to_excel(join(default_output, 'semanais','TABELA_testes_notifica_GRUPO01.xlsx'), index=False)
tabela_grupo01

In [ ]:
grupo02 = todos_exames.loc[(todos_exames['idade'] >= 10) & (todos_exames['idade'] <= 19)]

tabela_grupo02 = generate_table(grupo02)
tabela_grupo02.to_excel(join(default_output,'semanais','TABELA_testes_notifica_GRUPO02.xlsx'), index=False)
tabela_grupo02